# 🎯 Indian Language Identification (MMS-LID)

**Model:** `facebook/mms-lid-256`  
**Languages:** 12+ Indian languages including Hindi, Tamil, Telugu, Kannada, Bengali, etc.

---

## 📦 Installation (~1 min)

In [ ]:
!pip install transformers torch torchaudio soundfile librosa -q
print("✅ Installation complete!")

✅ Installation complete!


## 🎯 Language Detector Class

In [ ]:
import torch
import torchaudio
from transformers import Wav2Vec2ForSequenceClassification, AutoFeatureExtractor
from typing import Dict, List
import time
import os

class IndicLIDDetector:
    """
    Indian Language Identification using MMS-LID-256 model.
    Supports 12+ Indian languages with high accuracy.
    """

    # Map MMS language codes to common codes
    LANGUAGE_CODE_MAP = {
        'hin': 'hi', 'tam': 'ta', 'tel': 'te', 'kan': 'kn',
        'ben': 'bn', 'guj': 'gu', 'mal': 'ml', 'mar': 'mr',
        'pan': 'pa', 'ori': 'or', 'asm': 'as', 'urd': 'ur',
    }

    LANGUAGE_NAMES = {
        'hi': 'Hindi 🇮🇳', 'ta': 'Tamil 🎭', 'te': 'Telugu 📜', 'kn': 'Kannada 🏛️',
        'bn': 'Bengali 📚', 'gu': 'Gujarati 🪔', 'ml': 'Malayalam 🌴', 'mr': 'Marathi 🏔️',
        'pa': 'Punjabi 🌾', 'or': 'Odia 🎨', 'as': 'Assamese 🍵', 'ur': 'Urdu 📖',
    }

    INDIAN_LANGUAGES = {'hin', 'tam', 'tel', 'kan', 'ben', 'guj', 'mal', 'mar', 'pan', 'ori', 'asm', 'urd'}

    def __init__(self, model_id: str = "facebook/mms-lid-256", device: str = "auto"):
        self.model_id = model_id
        self.device = self._get_device(device)
        self.model = None
        self.processor = None
        self.id2label = None

    def _get_device(self, device: str) -> str:
        if device == "auto":
            return "cuda" if torch.cuda.is_available() else "cpu"
        return device

    def load_model(self):
        if self.model is not None:
            return
        print(f"🔄 Loading {self.model_id} on {self.device}...")
        start = time.time()
        self.processor = AutoFeatureExtractor.from_pretrained(self.model_id)
        self.model = Wav2Vec2ForSequenceClassification.from_pretrained(self.model_id)
        self.model = self.model.to(self.device)
        self.model.eval()
        self.id2label = self.model.config.id2label
        print(f"✅ Loaded in {time.time() - start:.1f}s ({len(self.id2label)} languages)")

    def load_audio(self, audio_path: str):
      import librosa
      waveform, sr = librosa.load(audio_path, sr=16000, mono=True)
      return torch.tensor(waveform), 16000

    def detect_language(self, audio_path: str, top_k: int = 5, indian_only: bool = True) -> Dict:
        self.load_model()
        start_time = time.time()

        waveform, sr = self.load_audio(audio_path)
        inputs = self.processor(waveform, sampling_rate=sr, return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            logits = self.model(**inputs).logits
            probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]

        predictions = sorted(
            [{'code': self.id2label[i], 'prob': float(p)} for i, p in enumerate(probs)],
            key=lambda x: x['prob'], reverse=True
        )

        if indian_only:
            indian_preds = [p for p in predictions if p['code'] in self.INDIAN_LANGUAGES]
            if indian_preds and indian_preds[0]['prob'] > 0.01:
                predictions = indian_preds

        best = predictions[0]
        best_code = self.LANGUAGE_CODE_MAP.get(best['code'], best['code'])

        result = {
            'language': best_code,
            'confidence': best['prob'],
            'top_predictions': predictions[:top_k],
            'time_ms': (time.time() - start_time) * 1000
        }

        # Display
        print(f"\n{'='*50}")
        print(f"📁 Audio: {audio_path}")
        print(f"{'='*50}")
        for p in predictions[:top_k]:
            code = self.LANGUAGE_CODE_MAP.get(p['code'], p['code'])
            name = self.LANGUAGE_NAMES.get(code, code)
            bar = '█' * int(p['prob'] * 20)
            print(f"{name:20} {p['prob']:6.1%} {bar}")
        print(f"{'='*50}")
        print(f"✅ DETECTED: {self.LANGUAGE_NAMES.get(best_code, best_code)} ({best['prob']:.1%})")
        print(f"⏱️  Time: {result['time_ms']:.0f}ms")

        return result

print("✅ IndicLIDDetector class ready!")

✅ IndicLIDDetector class ready!


## 🚀 Initialize Detector

In [ ]:
# Initialize (uses GPU if available)
detector = IndicLIDDetector(
    model_id="facebook/mms-lid-256",
    device="auto"
)
print(f"✅ Detector ready! Device: {detector.device}")

✅ Detector ready! Device: cuda


## 📤 Upload Audio Files

In [ ]:
from google.colab import files
import os

os.makedirs("samples", exist_ok=True)

print("📤 Upload your audio files (wav, mp3, m4a, flac):")
uploaded = files.upload()

for filename in uploaded.keys():
    os.rename(filename, f"samples/{filename}")
    print(f"✅ Saved: samples/{filename}")

📤 Upload your audio files (wav, mp3, m4a, flac):


Saving sample_142.wav to sample_142.wav
✅ Saved: samples/sample_142.wav


## 🎯 Detect Language

In [ ]:
# Process all uploaded files
audio_extensions = {'.wav', '.mp3', '.m4a', '.flac', '.ogg'}

audio_files = [
    f"samples/{f}" for f in os.listdir("samples")
    if os.path.splitext(f)[1].lower() in audio_extensions
]

print(f"\n🎤 Found {len(audio_files)} audio file(s)\n")

results = []
for audio_path in audio_files:
    result = detector.detect_language(audio_path)
    results.append(result)


🎤 Found 7 audio file(s)


📁 Audio: samples/sample_354.wav
Telugu 📜             100.0% ███████████████████
Tamil 🎭                0.0% 
Malayalam 🌴            0.0% 
Kannada 🏛️             0.0% 
Punjabi 🌾              0.0% 
✅ DETECTED: Telugu 📜 (100.0%)
⏱️  Time: 193ms

📁 Audio: samples/sample_22.wav
Telugu 📜              93.6% ██████████████████
Kannada 🏛️             3.6% 
Marathi 🏔️             0.6% 
Malayalam 🌴            0.6% 
Tamil 🎭                0.6% 
✅ DETECTED: Telugu 📜 (93.6%)
⏱️  Time: 209ms

📁 Audio: samples/sample_36.wav
Telugu 📜             100.0% ███████████████████
Tamil 🎭                0.0% 
Kannada 🏛️             0.0% 
Malayalam 🌴            0.0% 
Punjabi 🌾              0.0% 
✅ DETECTED: Telugu 📜 (100.0%)
⏱️  Time: 587ms

📁 Audio: samples/sample_432.wav
Marathi 🏔️             2.0% 
Assamese 🍵             0.5% 
Hindi 🇮🇳               0.4% 
Punjabi 🌾              0.4% 
Gujarati 🪔             0.3% 
✅ DETECTED: Marathi 🏔️ (2.0%)
⏱️  Time: 64ms

📁 Audio: samples/sample_1

## 🎤 Test Single Audio (Optional)

In [ ]:
# Test a specific audio file
# Replace with your file path

AUDIO_PATH = "samples/your_audio.wav"  # ← Change this!

if os.path.exists(AUDIO_PATH):
    from IPython.display import Audio, display
    display(Audio(AUDIO_PATH))
    result = detector.detect_language(AUDIO_PATH)
else:
    print(f"⚠️ File not found: {AUDIO_PATH}")

---
## 📊 Summary

| Feature | Value |
|---------|-------|
| Model | `facebook/mms-lid-256` |
| Languages | 12+ Indian languages |
| Speed | ~100-200ms per audio |
| Accuracy | High for Indian languages |